**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [ ]:
import tensorflow as tf
tf.VERSION

**Estimator**

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator/Estimator

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator

**model_fn**
```text
 |        model_fn: Model function. Follows the signature:
 |      
 |          * Args:
 |      
 |            * `features`: This is the first item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same.
 |            * `labels`: This is the second item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same (for multi-head models). If
 |                   mode is `ModeKeys.PREDICT`, `labels=None` will be passed. If
 |                   the `model_fn`'s signature does not accept `mode`, the
 |                   `model_fn` must still be able to handle `labels=None`.
 |            * `mode`: Optional. Specifies if this training, evaluation or
 |                   prediction. See `ModeKeys`.
 |            * `params`: Optional `dict` of hyperparameters.  Will receive what
 |                   is passed to Estimator in `params` parameter. This allows
 |                   to configure Estimators from hyper parameter tuning.
 |            * `config`: Optional configuration object. Will receive what is passed
 |                   to Estimator in `config` parameter, or the default `config`.
 |                   Allows updating things in your model_fn based on configuration
 |                   such as `num_ps_replicas`, or `model_dir`.
 |      
 |          * Returns:
 |            `EstimatorSpec`
 ```

In [ ]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    input_data = tf.concat([input_context, input_utterance], axis=0)
    input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
    input_length = tf.reshape(input_length, [-1])
    
    embeddings = tf.get_variable(
        'embeddings',
        shape=(vocab_size, embed_size),
        initializer=tf.random_uniform_initializer(-0.25, 0.25))

    input_embed = tf.nn.embedding_lookup(
        embeddings, input_data, name='input_embed')
        
    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
           'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=targets, logits=logits, reduction=tf.losses.Reduction.MEAN)
    
    return probs, loss

In [ ]:
def model_fn_train(features, labels, vocab_size, embed_size, hidden_size, learning_rate, optimizer):
    input_context = features['context']
    input_context_len = features['context_len']
    input_utterance = features['utterance']
    input_utterance_len = features['utterance_len']

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels)
    
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=learning_rate,
        clip_gradients=10.0,
        optimizer=optimizer)
    
    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.TRAIN,
        predictions=probs,
        loss=loss,
        train_op=train_op)

def model_fn_eval(features, labels, vocab_size, embed_size, hidden_size):
    input_context = []
    input_context_len = []
    input_utterance = []
    input_utterance_len = []
    labels_ = []
    
    context = features['context']
    context_len = features['context_len']

    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['utterance'])
    input_utterance_len.append(features['utterance_len'])
    labels_.append(tf.ones_like(context_len))
    
    for i in range(9):
        input_context.append(context)
        input_context_len.append(context_len)
        input_utterance.append(features['distractor_{}'.format(i)])
        input_utterance_len.append(features['distractor_{}_len'.format(i)])
        labels_.append(tf.zeros_like(context_len))

    input_context = tf.concat(input_context, axis=0)
    input_context_len = tf.concat(input_context_len, axis=0)
    input_utterance = tf.concat(input_utterance, axis=0)
    input_utterance_len = tf.concat(input_utterance_len, axis=0)
    labels_ = tf.concat(labels_, axis=0)

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels_)
    
    split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
    predictions = tf.concat(split_probs, axis=1)
    predictions_2 = predictions[:, :2]
    
    recall_at_1_2 = tf.metrics.recall_at_k(labels=labels, predictions=predictions_2, k=1)
    recall_at_1_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)
    recall_at_2_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=2)
    recall_at_5_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)
    
    eval_metric_ops = {
        'recall_at_1_2': recall_at_1_2,
        'recall_at_1_10': recall_at_1_10,
        'recall_at_2_10': recall_at_2_10,
        'recall_at_5_10': recall_at_5_10,
    }

    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        predictions=probs,
        loss=loss,
        train_op=None,
        eval_metric_ops=eval_metric_ops
    )

def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = params['learning_rate']
        optimizer =  params['optimizer']
        return model_fn_train(
            features, labels, vocab_size, embed_size, hidden_size, learning_rate, optimizer)
    if mode == tf.estimator.ModeKeys.EVAL:
        return model_fn_eval(features, labels, vocab_size, embed_size, hidden_size)
    
    return None

**Input**

In [ ]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def features_train(vector_length):
    return [
        tf.feature_column.numeric_column(
            key='context', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='context_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='label', shape=1, dtype=tf.int64),
    ]

def features_eval(vector_length):
    features = []
    keys = ['context', 'utterance']
    keys += ['distractor_{}'.format(i) for i in range(9)]
    for key in keys:
        features += [
            tf.feature_column.numeric_column(
                key=key, shape=vector_length, dtype=tf.int64),
            tf.feature_column.numeric_column(
                key=key + '_len', shape=1, dtype=tf.int64),
        ]
    return features

def _input_reader(name, filenames, features, batch_size, num_epochs):
    example_features = tf.feature_column.make_parse_example_spec(features)
    return tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        randomize_input=True,
        queue_capacity=200000 + batch_size * 10,
        name='read_batch_record_features_' + name
    )


def input_train(name, filenames, features, batch_size, num_epochs=None):
    batch_example = _input_reader(name, filenames, features, batch_size, num_epochs)
    batch_target = batch_example.pop('label')
    return batch_example, batch_target

def input_eval(name, filenames, features, batch_size, num_epochs=None):
    batch_example = _input_reader(name, filenames, features, batch_size, num_epochs)
    batch_target = tf.zeros_like(batch_example['context_len'])
    return batch_example, batch_target

**Training**

In [ ]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
TRAIN_TFR = os.path.join(DATA_DIR, 'train.tfrecords')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

def has_file(file):
    if not os.path.isfile(file):
        raise Exception('File not found: {}'.format(file))

has_file(VOCAB_BIN)
has_file(TRAIN_TFR)
has_file(VALID_TFR)
has_file(TEST_TFR)

In [ ]:
vocab = VocabularyAdapter(VOCAB_BIN)
train_features = features_train(vocab.vector_length)
eval_features = features_eval(vocab.vector_length)

In [ ]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 200,
    'learning_rate': 0.001,
    'optimizer': 'Adam',
    'batch_size': 256,
    'num_epochs': 2,
}

input_fn_train = lambda: input_train('train', [TRAIN_TFR], train_features, params['batch_size'], 1)
input_fn_valid = lambda: input_eval('valid', [VALID_TFR], eval_features, 16, 1)
input_fn_test = lambda: input_eval('test', [TEST_TFR], eval_features, 16, 1)

In [ ]:
import shutil

def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

MODEL_DIR = os.path.join(HOME_DIR, 'model')

remove_dir(MODEL_DIR)

In [ ]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

In [ ]:
def epoch_range(i, num_epochs):
    range_start = i * num_epochs + 1
    range_end = range_start + num_epochs
    return range(range_start, range_end)

In [ ]:
%%time

for epoch in epoch_range(0, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid, steps=None)
    print()

In [ ]:
%%time

estimator.evaluate(input_fn_test, steps=None)